<a href="https://colab.research.google.com/github/chandrakar-shubham/Shacklabs_job_assignment-Regression-model-and-Similar-texts-/blob/main/Shack_lab_assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Assigment 2 ---> Shack labs

Given two csv file contains amazon data and flipkart data respectively.

Objective: To create new dataframe containing similar product of flipkart and amazon with information of selling price in that platform and discounted price in that platform.

#Data dictionary

df1 = dataframe made from amazon data

df2 = dataframe made from flipkart data



In [53]:
#importing basic libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [56]:
#importing sentence transformer , cosine similarity from torch ilbrary

from sentence_transformers import SentenceTransformer, util
import torch
from torch.nn.functional import cosine_similarity

In [54]:
from torch.nn.functional import cosine_similarity
#similar_prod_indexes = []

def similar_products(model_name,data1,data2,path_to_export):


  '''take model, two dataframe and path to export as input,
  return final database based on similarity,
  and export data to a location'''


  def embedded_model(model_name,data1,data2):
    '''create model of sentence transformer, take input as two dataframe and return cosine similarity'''
    # Creating a model of sentence transformer class
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Computing embedding for both amazon and flipkart product 
    df1_embedding = model.encode(data1.product_name,convert_to_tensor=True)                      #.to_numpy()
    df2_embedding = model.encode(data2.product_name,convert_to_tensor=True)

    # Computing cosine similarities
    cosine_sim_scores = util.cos_sim(df1_embedding,df2_embedding)

    return cosine_sim_scores


  def store_similar_prod_index(cosine_sim_scores,df1,df2):
    '''input tensor of cosine scores, two data frame and store index of similar products'''
    
    length = len(df1)
    
    if len(df1) > len(df2):
      length =len(df2)
    else:
      length = len(df1)

    # Storing the index of a similar Flipkart product corresponding to each Amazon product 
    similar_prod_indexes = []
    
    for i in range(length):
      max_score_idxs = torch.topk(cosine_sim_scores[i],k=5,largest=True,sorted=True).indices
      for idx in max_score_idxs:
        if idx != i:
          similar_prod_indexes.append(idx)
          break
    return similar_prod_indexes
  
  
  def create_sim_list(similar_prod_indexes):
    
    '''create list of similar product index'''

    #similar_prod_indexes = [x.item() for x in similar_prod_indexes]
    sim_index = [x.item() for x in similar_prod_indexes]
    
    return sim_index

  
  def extract_similar_data(similar_prod_indexes,data1,data2):

    '''input two dataframe and list of similar product index and extract details from dataframe'''
    
    # Extracting the details such as retail price and discounted price for both Flipkart and Amazon similar products
    flipkart_prod_data = []
    amazon_prod_data = []
    
    for idx, prod_name in enumerate(df2.product_name):
      flipkart_prod_data.append(df2.iloc[similar_prod_indexes[idx]])
      amazon_prod_data.append(df1.iloc[idx])
      #print(idx,prod_name)

    return amazon_prod_data, flipkart_prod_data

  def create_dataframe(amazon_prod_data, flipkart_prod_data):

    '''create dataframe two dataframe by on basis of similarity to data'''
    
    flipkart_prod_data = pd.DataFrame(flipkart_prod_data)
    flipkart_prod_data.columns = ['Product name in Flipkart','Retail Price in Flipkart','Discounted Price in Flipkart']
    
    amazon_prod_data = pd.DataFrame(amazon_prod_data)
    amazon_prod_data.columns = ['Product name in Amazon','Retail Price in Amazon','Discounted Price in Amazon']

    amazon_prod_data['Retail Price in Amazon'] = amazon_prod_data['Retail Price in Amazon'].astype(np.float64)
    amazon_prod_data['Discounted Price in Amazon'] = amazon_prod_data['Discounted Price in Amazon'].astype(np.float64)

    flipkart_prod_final_data = flipkart_prod_data.copy()
    flipkart_prod_final_data = flipkart_prod_final_data.reset_index(drop=True)
    flipkart_prod_final_data.head()

    return amazon_prod_data,flipkart_prod_final_data



  def export_data_csv(path_to_export,amazon_prod_data,flipkart_prod_final_data):

    '''create final database and export data in form of csv'''
    
    final_prod_data = pd.concat([flipkart_prod_final_data,amazon_prod_data],axis=1)
    final_prod_data.to_csv(path_to_export + 'final_result.csv')

    return final_prod_data

  cosine_scores = embedded_model(model_name,data1,data2)

  similar_pro_index = store_similar_prod_index(cosine_scores,data1,data2)

  similar_prod_indexes = create_sim_list(similar_pro_index)

  amazon_data,flip_data = extract_similar_data(similar_prod_indexes,data1,data2)

  amazon_df,flip_df = create_dataframe(amazon_data, flip_data)

  final_df = export_data_csv(path_to_export,amazon_df,flip_df)

  return final_df

In [55]:
#installing sentence transformer library

!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [57]:
# path of the file

path_amzon = '/content/drive/MyDrive/Internship and job assignment/Shack labs/assignment 2 data/amz_com-ecommerce_sample.csv'
path_flip= '/content/drive/MyDrive/Internship and job assignment/Shack labs/assignment 2 data/flipkart_com-ecommerce_sample.csv'

#importing dataset in pandas

df1 =  pd.read_csv(path_amzon,encoding = 'ISO-8859-1')
df2 = pd.read_csv(path_flip,encoding = 'ISO-8859-1')

#Data Wrangling

In [58]:
#Top 5 rows in amazon dataframe

df1.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,982,438,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32143,29121,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,991,551,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,694,325,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,208,258,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [59]:
#Bottom 5 row in amazon df    

df1.tail()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
19995,7179d2f6c4ad50a17d014ca1d2815156,2015-12-01 10:15:43 +0000,http://www.flipkart.com/walldesign-small-vinyl...,WALLDESIGN SMALL VINYL STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1498,876,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ..."
19996,71ac419198359d37b8fe5e3fffdfee09,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,WALLMANTRA LARGE VINYL STICKERS STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1415,1424,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ..."
19997,93e9d343837400ce0d7980874ece471c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,ELITE COLLECTION MEDIUM ACRYLIC STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1284,1196,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19998,669e79b8fa5d9ae020841c0c97d5e935,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,ELITE COLLECTION MEDIUM ACRYLIC STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE8YSVEPPCZ42Y,1492,1364,"[""http://img5a.flixcart.com/image/sticker/4/2/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19999,cb4fa87a874f715fff567f7b7b3be79c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,ELITE COLLECTION MEDIUM ACRYLIC STICKER,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE88KN9ZDSGZKY,1484,1247,"[""http://img6a.flixcart.com/image/sticker/z/k/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."


In [60]:
#checking info in amazon dataframe

df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   uniq_id                  20000 non-null  object
 1   crawl_timestamp          20000 non-null  object
 2   product_url              20000 non-null  object
 3   product_name             20000 non-null  object
 4   product_category_tree    20000 non-null  object
 5   pid                      20000 non-null  object
 6   retail_price             20000 non-null  int64 
 7   discounted_price         20000 non-null  int64 
 8   image                    19997 non-null  object
 9   is_FK_Advantage_product  20000 non-null  bool  
 10  description              19998 non-null  object
 11  product_rating           20000 non-null  object
 12  overall_rating           20000 non-null  object
 13  brand                    14136 non-null  object
 14  product_specifications   19986 non-nul

In [61]:
# Top 5 row in flipkart dataframe

df2.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [62]:
#bottom 5 rows in filpkart dataframe

df2.tail()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
19995,7179d2f6c4ad50a17d014ca1d2815156,2015-12-01 10:15:43 +0000,http://www.flipkart.com/walldesign-small-vinyl...,WallDesign Small Vinyl Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7KFJAKSTDY9G,1500.0,730.0,"[""http://img6a.flixcart.com/image/wall-decorat...",False,Buy WallDesign Small Vinyl Sticker for Rs.730 ...,No rating available,No rating available,WallDesign,"{""product_specification""=>[{""key""=>""Number of ..."
19996,71ac419198359d37b8fe5e3fffdfee09,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,Wallmantra Large Vinyl Stickers Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE9F5URNQGJCGH,1429.0,1143.0,"[""http://img6a.flixcart.com/image/sticker/z/g/...",False,Buy Wallmantra Large Vinyl Stickers Sticker fo...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ..."
19997,93e9d343837400ce0d7980874ece471c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE7VAYDKQZEBSD,1299.0,999.0,"[""http://img5a.flixcart.com/image/sticker/b/s/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19998,669e79b8fa5d9ae020841c0c97d5e935,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE8YSVEPPCZ42Y,1499.0,1199.0,"[""http://img5a.flixcart.com/image/sticker/4/2/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."
19999,cb4fa87a874f715fff567f7b7b3be79c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/elite-collection-mediu...,Elite Collection Medium Acrylic Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE88KN9ZDSGZKY,1499.0,999.0,"[""http://img6a.flixcart.com/image/sticker/z/k/...",False,Buy Elite Collection Medium Acrylic Sticker fo...,No rating available,No rating available,Elite Collection,"{""product_specification""=>[{""key""=>""Number of ..."


In [63]:
#checking information of flipkart dataframe

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

Keeping only relevant columns

In [64]:
# retain only relevant information

to_retain =['product_name','retail_price','discounted_price']
df1 = df1[to_retain]
df2 = df2[to_retain]

Check for duplicates

In [65]:
len(df1[df1.duplicated()])

17

In [66]:
len(df2[df2.duplicated()])

4425

In [67]:
#df1 = df1[~df1.duplicated()]
#df2 = df2[~df2.duplicated()]

Check for null values

In [68]:
df1.isnull().sum()

product_name        0
retail_price        0
discounted_price    0
dtype: int64

In [69]:
df2.isnull().sum()

product_name         0
retail_price        78
discounted_price    78
dtype: int64

In [70]:
df2.fillna(0,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [71]:
df1.shape

(20000, 3)

In [72]:
df2.shape

(20000, 3)

In [73]:
#df1['pn1']=df1['product_name'].str.lower()
#df2['pn2']=df2['product_name'].str.lower()

In [74]:
df1.head()

,product_name,retail_price,discounted_price
0,Alisha Solid Women's Cycling Shorts,982,438
1,FabHomeDecor Fabric Double Sofa Bed,32143,29121
2,AW Bellies,991,551
3,Alisha Solid Women's Cycling Shorts,694,325
4,Sicons All Purpose Arnica Dog Shampoo,208,258


In [75]:
df2.head()

,product_name,retail_price,discounted_price
0,Alisha Solid Women's Cycling Shorts,999.0,379.0
1,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0
2,AW Bellies,999.0,499.0
3,Alisha Solid Women's Cycling Shorts,699.0,267.0
4,Sicons All Purpose Arnica Dog Shampoo,220.0,210.0


In [76]:
df1.shape

(20000, 3)

In [77]:
df2.shape

(20000, 3)

#Sentence transformation

In [78]:

#name of the model used for sentence transformation
model_name = SentenceTransformer('all-MiniLM-L6-v2')

#amazon data after data wrangling as data1
data1 = df1

#flipkart data after data wrangling as data2
data2 = df2

#path where final csv file is to be exported
path_to_export ='/content/drive/MyDrive/Internship and job assignment/Shack labs/'



In [79]:
#create and export final dataframe by using sentence transformer model

#calling similar products function
final_df = similar_products(model_name,data1,data2,path_to_export)

#printing final df
final_df

,Product name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart,Product name in Amazon,Retail Price in Amazon,Discounted Price in Amazon
0,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,982.0,438.0
1,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32143.0,29121.0
2,i-Sports Bellies,899.0,399.0,AW Bellies,991.0,551.0
3,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,694.0,325.0
4,Sicons Conditioning Conditoner Dog Shampoo,110.0,100.0,Sicons All Purpose Arnica Dog Shampoo,208.0,258.0
...,...,...,...,...,...,...
19995,WallDesign Small Vinyl Sticker,1220.0,600.0,WALLDESIGN SMALL VINYL STICKER,1498.0,876.0
19996,Wallmantra Large Vinyl Stickers Sticker,727.0,582.0,WALLMANTRA LARGE VINYL STICKERS STICKER,1415.0,1424.0
19997,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284.0,1196.0
19998,Elite Collection Medium Acrylic Sticker,1299.0,999.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492.0,1364.0


In [80]:
#top 10 rows of final df
final_df.head(10)

,Product name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart,Product name in Amazon,Retail Price in Amazon,Discounted Price in Amazon
0,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,982.0,438.0
1,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32143.0,29121.0
2,i-Sports Bellies,899.0,399.0,AW Bellies,991.0,551.0
3,Alisha Solid Women's Cycling Shorts,999.0,379.0,Alisha Solid Women's Cycling Shorts,694.0,325.0
4,Sicons Conditioning Conditoner Dog Shampoo,110.0,100.0,Sicons All Purpose Arnica Dog Shampoo,208.0,258.0
5,Aaishwarya Glim Glam Crystal Crystal Alloy Nec...,1450.0,599.0,Eternal Gandhi Super Series Crystal Paper Weig...,427.0,473.0
6,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,1198.0,602.0
7,FabHomeDecor Fabric Double Sofa Bed,32157.0,22646.0,FabHomeDecor Fabric Double Sofa Bed,32137.0,28664.0
8,Provogue Corporate Casuals,2485.0,2485.0,"dilli bazaaar Bellies, Corporate Casuals, Casuals",682.0,385.0
9,Alisha Solid Women's Cycling Shorts,699.0,267.0,Alisha Solid Women's Cycling Shorts,1197.0,542.0


In [81]:
#bottom 10 rows of final df
final_df.tail(10)

,Product name in Flipkart,Retail Price in Flipkart,Discounted Price in Flipkart,Product name in Amazon,Retail Price in Amazon,Discounted Price in Amazon
19990,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1289.0,1293.0
19991,WallDesign Small Vinyl Sticker,1500.0,730.0,WALLDESIGN SMALL VINYL STICKER,1212.0,684.0
19992,999store Medium Paper Sticker,999.0,599.0,999STORE MEDIUM PAPER STICKER,989.0,713.0
19993,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1282.0,1164.0
19994,Wallmantra Large Vinyl Stickers Sticker,727.0,582.0,WALLMANTRA LARGE VINYL STICKERS STICKER,709.0,721.0
19995,WallDesign Small Vinyl Sticker,1220.0,600.0,WALLDESIGN SMALL VINYL STICKER,1498.0,876.0
19996,Wallmantra Large Vinyl Stickers Sticker,727.0,582.0,WALLMANTRA LARGE VINYL STICKERS STICKER,1415.0,1424.0
19997,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1284.0,1196.0
19998,Elite Collection Medium Acrylic Sticker,1299.0,999.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1492.0,1364.0
19999,Elite Collection Medium Acrylic Sticker,1499.0,1199.0,ELITE COLLECTION MEDIUM ACRYLIC STICKER,1484.0,1247.0


#Conclusion: 

Sentence transformer is best model for arranging similar product together.